# Embedding and Imputation on Ramani et al. scHi-C

## Notes

This tutorial uses the new API of Higashi (wrapping all functions of Higashi into the Higashi() class).
The old API of Higashi will still be supported and maintained).
Please check changelog for the current status of migration from the old API to the new API.

## Preparation

### Download input files
Download the demo data from the following link (Ramani et al.)
https://drive.google.com/drive/folders/1S0KOMAj60MxQP6mgPV1OKjn_J-lVpzKM?usp=sharing

The dataset contains 620 cells from the ML1/ML3 library of the Ramani et al. dataset.

Change the file path in the corresponding JSON file according to the location of the downloaded files.

### Install Higashi

1. install pytorch>=1.8.0 with cuda support when available.
2. `conda install -c ruochiz higashi`
(Although higashi would install pytorch when needed, there is no guarantee that it will install the correct version with cuda support. It is recommended to install pytorch separately before higashi.)

## Start running Higashi¶

### 1. Import package, set the path to the configuration JSON file.¶

In [2]:
cd ..

/home/mscs/congfeng4/GuidedStudy/Code/Higashi


In [3]:
from higashi.Higashi_wrapper import *
config = "./Data/config_ramani.JSON"
higashi_model = Higashi(config)

### 2. Process data for higashi model

In [ ]:
# higashi_model.process_data()

: 

### 3. Prep the higashi model for training and imputation & Stage 1 training

In [ ]:
higashi_model.prep_model()
# Stage 1 training
higashi_model.train_for_embeddings()

cpu_num 6
setting to gpu:0
training on data from: ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']
total_sparsity_cell 0.025731658257764963
no contractive loss
batch_size 256
Node type num [620 250 244 199 192 181 172 160 147 142 136 136 134 116 108 103  91  82
  79  60  64  49  52 156] [ 620  870 1114 1313 1505 1686 1858 2018 2165 2307 2443 2579 2713 2829
 2937 3040 3131 3213 3292 3352 3416 3465 3517 3673]
start making attribute



loss 0.5252807140350342 loss best 0.5252290368080139 epochs 125

initializing data generator



initializing data generator



First stage training
[ Epoch 0 of 60 ]


- (Train)   bce:  0.6200, mse:  0.6536,  acc: 51.117 %, pearson: 0.332, spearman: 0.301, elapse: 239.278 s


...

- (Valid) bce:  0.5707,  acc: 79.375 %,pearson: 0.473, spearman: 0.418,elapse: 0.059 s
update_rate: 0.009473	0.114868	
[ Epoch 30 of 60 ]


- (Train)   bce:  0.5619, mse:  0.6677,  acc: 80.381 %, pearson: 0.498, spearman: 0.433, elapse: 116.966 s


- (Valid) bce:  0.5691,  acc: 80.505 %,pearson: 0.479, spearman: 0.419,elapse: 0.060 s
update_rate: 0.012813	0.143748	
[ Epoch 31 of 60 ]


- (Train)   bce:  0.5618, mse:  0.6695,  acc: 80.454 %, pearson: 0.501, spearman: 0.433, elapse: 118.547 s


- (Valid) bce:  0.5795,  acc: 80.932 %,pearson: 0.479, spearman: 0.421,elapse: 0.061 s
update_rate: 0.008810	0.085481	
[ Epoch 32 of 60 ]


### 4. Stage 2 training and imputation without neighbor information

In [ ]:
higashi_model.train_for_imputation_nbr_0()
higashi_model.impute_no_nbr()

### 5. Stage 3 training and imputation with neighbor information

In [ ]:
higashi_model.train_for_imputation_with_nbr()
higashi_model.impute_with_nbr()

### 5. Visulizing embedding results

In [ ]:
# Visualize embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from umap import UMAP
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

cell_type = higashi_model.label_info['cell type']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
vec = PCA(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
vec = UMAP(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.show()



### 6. Visualizing imputation results

In [ ]:
count = 0
fig = plt.figure(figsize=(6, 2*5))
for id_ in np.random.randint(0, 620, 5):
    ori, nbr0, nbr5 = higashi_model.fetch_map("chr3", id_)
    count += 1
    ax = plt.subplot(5, 3, count * 3 - 2)
    ax.imshow(ori.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(ori.data, 0.6))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("raw")

    ax = plt.subplot(5, 3, count * 3 - 1)
    ax.imshow(nbr0.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr0.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=0")

    ax = plt.subplot(5, 3, count * 3)
    ax.imshow(nbr5.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr5.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=5")

plt.tight_layout()